# Advent of Code

The docstrings are pretty bad.

In [2]:
import os
import re
import math
from datetime import datetime
from typing import List, Tuple, Optional, Any, Callable
import textwrap

from pydantic.dataclasses import dataclass, Field
import requests as req
import pandas as pd
import numpy as np
import numpy.ma as ma

## Helper Functions

In [3]:
def print_solutions(a1sol, a2sol):
    print(textwrap.dedent(f"""
        a1_solution: {a1sol}
        a2_solution: {a2sol}
    """))

---
## Advent Day 1

In [3]:
data_01 = np.genfromtxt('a01.csv', delimiter=',')

def advent_01a(data: np.ndarray = data_01) -> int:
    """Gives a solution to https://adventofcode.com/2021/day/1, part 1."""
    return np.sum(np.diff(data) > 0)

def advent_01b(data: np.ndarray = data_01) -> int:
    """Gives a solution to https://adventofcode.com/2021/day/1, part 2."""
    lag_array = np.array([
        [data[idx], data[idx + 1], data[idx + 2]] 
        for idx in range(len(data) - 2)
    ])
    lag_array_sum_across = np.sum(lag_array, axis=1)
    return np.sum(np.diff(lag_array_sum_across) > 0)

print_solutions(advent_01a(), advent_01b())


a1_solution: 1696
a2_solution: 1737



---
## Advent Day 2

In [4]:
@dataclass
class XYCoord:
    """XYCoordinate with addition and scalar multiplication."""
    x: int
    y: int
    
    def __add__(self, coord):
        return XYCoord(self.x + coord.x, self.y + coord.y)

    def __mul__(self, other: int):
        return XYCoord(self.x * other, self.y * other)
    
    def __rmul__(self, other: int):
        return self.__mul__(other)
    
    
direction_map = {
    "forward": XYCoord(1, 0),
    "down": XYCoord(0, 1),
    "up": XYCoord(0, -1)
}

def parse_directions_part_1(data: List[str]) -> XYCoord:
    """Parses directions a la https://adventofcode.com/2021/day/2 ."""
    current_loc = XYCoord(0, 0)
    aim = 0
    
    # Split the direction "forward 5", map "forward" to XYCoord, mult by int("5"),
    # and add to current loc.
    for row in data:
        direction, amount = row.split(" ")
        amount = int(amount)
        current_loc += amount * direction_map[direction]

    return current_loc    

def parse_directions_part_2(data: List[str]) -> XYCoord:
    """Parses directions a la https://adventofcode.com/2021/day/2 ."""
    current_loc = XYCoord(0, 0)
    aim = 0
    
    # Split the direction "forward 5", map "forward" to XYCoord, mult by int("5"),
    # and add to current loc.
    for row in data:
        direction, amount = row.split(" ")
        amount = int(amount)        
                
        if direction == "down":
            aim += amount
        elif direction == "up":
            aim -= amount
        elif direction == "forward":
            current_loc += XYCoord(amount, amount * aim)

    return current_loc
        
    
# MAIN
with open("a02.csv", "r") as f:
    data = [line.strip() for line in f.readlines() if line]

def advent_02a(data: List[str] = data) -> int:
    """Solves Advent 2a."""
    dirs = parse_directions_part_1(data)
    return dirs.x * dirs.y

def advent_02b(data: List[str] = data) -> int:
    """Solves Advent 2b."""
    dirs = parse_directions_part_2(data)
    return dirs.x * dirs.y

print_solutions(advent_02a(), advent_02b())


a1_solution: 2117664
a2_solution: 2073416724



---
## Advent Day 3

In [77]:
with open("a03.csv", "r") as f:
    # Open file, create matrix of rows / cols.
    data_03 = [line.strip() for line in f.readlines()]
    
def binary_string_list_to_2D_array(data: List[str]) -> np.ndarray:
    """ Transforms a list of same-sized binary digits into a matrix with
    each value as a row. """
    return np.array([list(map(int, list(row))) for row in data])

def a03a(data: List[str]) -> int:
    """Solves https://adventofcode.com/2021/day/3."""
    data = binary_string_list_to_2D_array(data)
    binary_string_length = len(data[0])
    
    def mode_is_one(row: np.array) -> str:
        """Returns True if 1 is more frequent or THE SAME FREQUENCY as 0."""
        return sum(row) >= len(row) / 2
    
    def int_from_bin_list(s: List[int]) -> int:
        """Returns the int value of a list of binary values."""
        return int("".join(map(str, s)), base=2)
        
    gamma_rate = int_from_bin_list(
        [1 if mode_is_one(data[:, idx]) else 0 for idx in range(len(data[0]))]
    )
    
    epsilon_rate = int_from_bin_list(
        ["0" if mode_is_one(data[:, idx]) else "1" for idx in range(len(data[0]))]
    )

    return gamma_rate * epsilon_rate


class A03b:
    def __init__(self, data: List[str]):
        """Solves https://adventofcode.com/2021/day/3."""
        self.data = binary_string_list_to_2D_array(data)
        self.oxygen_generator_rating = self.data.copy()
        self.co2_scrubber_rating = self.data.copy()
        
        # Main
        self.oxy = self.find_rating(self.oxygen_generator_rating, self.oxy_mode_return)
        self.co2 = self.find_rating(self.co2_scrubber_rating, self.co2_mode_return)

    @staticmethod
    def binary_string_list_to_2D_array(data: List[str]) -> np.ndarray:
        """ Transforms a list of same-sized binary digits into a matrix with
        each value as a row. """
        return np.array([list(map(int, list(row))) for row in data])
        
    def int_from_bin_list(self, s: List[int]) -> int:
        """Returns the int value of a list of binary values."""
        return int("".join(map(str, s)), base=2)
    
    # NOTE: This part is gross.  I should have done something like calculate 
    # s >= data_size / 2 then made oxy / co2 return based on that value...
    def oxy_mode_return(self, s: int, data_size: int) -> bool:
        return 1 if s >= data_size / 2 else 0
    
    def co2_mode_return(self, s: int, data_size: int) -> bool:
        return 0 if s >= data_size / 2 else 1
    
    def find_rating(self, data: np.ndarray, mode_return: Callable) -> np.ndarray:
        rating = data.copy()
        for idx in range(len(rating[:, 0])):
            if len(rating) == 1:
                break
            s = sum(rating[:, idx])
            desired_bit = mode_return(s, len(rating[:, 0]))
            rating = np.array([row for row in rating if row[idx] == desired_bit])
        return self.int_from_bin_list(rating[0])

a03b = A03b(data_03)
print_solutions(a03a(data_03), a03b.oxy * a03b.co2)


a1_solution: 3912944
a2_solution: 4996233



---
## Advent Day 4

In [83]:
class Board:
    def __init__(self, board: np.ndarray):
        self.board = board.copy()
        self.called = np.zeros_like(self.board)
        self.size: int = self.board.shape[0]  # Square
        
    def mark_board(self, lookup_num: int) -> None:
        for row in range(self.size):
            for col in range(self.size):
                if self.board[row, col] == lookup_num:
                    self.called[row, col] = 1
    
    def check_card(self) -> bool:
        for idx in range(self.size):
            if ((sum(self.called[idx, :]) == 5) or
                (sum(self.called[:, idx]) == 5)):
                return True
        return False
        
    def reset_card(self):
        self.called = np.zeros_like(self.board)

In [84]:
def initialize() -> Tuple[Board, np.ndarray]:
    with open("a04_boards.csv", "r") as boards_f:
        boards_txt = boards_f.read()

    boards_raw = [board.split("\n") for board in boards_txt.split("\n\n")]
    boards_raw_parsed = []
    for board in boards_raw:
        board_parsed = []
        for row in board:
            board_parsed.append([int(i) for i in row.split(" ") if i])
        boards_raw_parsed.append(board_parsed)
    boards_raw_parsed = np.array(boards_raw_parsed)

    with open("a04_calls.csv", "r") as calls_f:
        calls = np.genfromtxt(calls_f, delimiter=",", encoding="utf-8")

    boards = []
    for board in boards_raw_parsed:
        boards.append(Board(board))

    return boards, calls

def play_bingo() -> int:
    boards, calls = initialize()
        
    def calculate_winning_score(board: Board, call: int):
        mx = ma.masked_array(board.board, mask=board.called)
        return int(call * mx.sum())
        
    for call in calls:
        for board in boards:
            board.mark_board(call)
            if board.check_card():
                return calculate_winning_score(board, call)
            
def lose_at_bingo() -> int:
    boards, calls = initialize()
    non_winning_boards = boards.copy()
    
    def calculate_winning_score(board: Board, call: int):
        mx = ma.masked_array(board.board, mask=board.called)
        return int(call * mx.sum())
        
    winning_board = None
    for call in calls:
        for board in non_winning_boards:
            board.mark_board(call)

        non_winning_boards = [board for board in boards if not board.check_card()]
        if len(non_winning_boards) == 1:
            winning_board = non_winning_boards[0]
        
        if len(non_winning_boards) == 0:
            # The last board won!
            return calculate_winning_score(winning_board, call)

print_solutions(play_bingo(), lose_at_bingo())


a1_solution: 38913
a2_solution: 16836



---
## Advent Day 5